In [ ]:
import os
import lephare as lp
import numpy as np
import matplotlib

%matplotlib inline
from matplotlib import pylab as plt

# Empirical Model

## Principle

This method allows to include the dominant emission lines in any kind of templates (even the empirical templates like the COSMOS_SED, or CWW).

It concentrates only on few emission lines which have potentially the largest impact on the photometry, i.e. 
Lyα λ1216A, [OII] λ3727A, Hβ λ4861A, [OIII]a λ4959, [OIII]b 5007, Hα λ6563

The principle is to use the M(NUV) for a given template to estimate the associated emission line flux (e.g. Halpha). We use the Kennicutt relations, corrected from dust attenuation.
The original relation in Ilbert et al. (2009) was using the relation between M(MNUV) and [OII] from Kennicutt (1998).
We use now an updated relation from Kennicutt & Evans (2012) which links M(NUV) and f(Ha) expected for a given redshifted template.\
log f(Halpha) = -0.4*M(NUV)  -6.224 - DM(z)/2.5 

Then, we assume constant ratios to predict the other lines.


## Relation between M(NUV) and f(Halpha)


The two original relations from Kennicutt & Evans (2012) are the following:
- log(SFR)=log(L(NUV))-43.17 with L(NUV) in erg/s nuLnu
- log(SFR)=log(L(Halpha))-41.27 with L(Halpha) in erg/s

We need to change the unit:
- log(SFR)=log(LNUV)-28.056 with L(NUV) in erg/s/Hz

Then we convert L(Halpha) erg.s-1 intrinsic luminosity  into f(Halpha) erg s-1 cm-2 observed flux
- log f(Halpha) = log L(Halpha) - 2*log( d ) - 50.077   with d in Mpc
- log f(Halpha) = log L(Halpha) - DM(z)/2.5  - 40.077   with d in Mpc
with D(z) being the distance modulus.

Then
- log(L(NUV))-28.056=log(f(Halpha))+DM(z)/2.5+ 40.077-41.27
- log(L(NUV))=log(f(Halpha))+DM(z)/2.5+ 26.863

Since M(AB) = -2.5 log(L) + 51.59 with L in erg.s-1.Hz-1
-  -0.4 * ( M(NUV) - 51.59 )   =  log(f(Halpha))+DM(z)/2.5+ 26.863
-  -0.4*M(NUV)  -6.224 - DM(z)/2.5 =   log f(Halpha)

In the code, Hbeta is now the reference to be consistent with the other methods. Therefore:\
f(Hbeta)=10^(-0.4*M(NUV)  -6.224 - DM(z)/2.5) / 2.85


## Strength ratios between the emission lines


We use constant ratios to establish the flux in the other emission lines. Such method is clearly prone to large uncertainties and can only provide a crude estimate of the emission line fluxes.
The ratio are internally stored in emission_lines.h.

### Original ratios

A first set of ratio is stored in empirical_ratio_ori. That's the original ratios described in section 3.2 of Ilbert et al. (2009) 
"we adopt intrinsic, unextincted flux ratios of [O IIIb/O II] = 0.36; [Hβ/O II] = 0.61; [Hα/O II] = 1.77 and [Lyα/O II] = 2 (McCall et al. 1985, Moustakas et al. 2006, Mouhcine et al. 2005, Kennicutt 1998)."  
Given that Hbeta is the reference, and that we split [OII] as a doublet, we get:
- OIIa 0.81
- OIIb 0.81
- Hbeta 1
- OIIIa 0.21
- OIIIb 0.59
- Halpha 2.90
  
For Lyman alpha, we rather use the intrinsic ratio of 22.2. But dust attenuation and opacity will reduce strongly this flux emission. We also apply the Hayes et al (2011) relation to derive the escape fraction.

### Updated ratios

A second set of empirical ratios is also included in the code and used by default
(harcoded in empirical_ratio from emission_lines.h).
- OIIa 1.425
- OIIb 1.425
- Hbeta 1
- OIIIa 0.38
- OIIIb 1.14
- Halpha 2.85
- NII 0.86

The updated version of the ratios is explained hereafter (mainly coming from a work by Iary Davidzon for his 2017 paper):


*Ha/[OII]*

The ratio Halpha/[OII]=1.77 was coming from Kennicutt (1998) but it seems that [OII] was not corrected for dust at 3727 A. 
According to the Kewley, Geller & Jansen 2004 theoretical calibration, the Ha/[OII] ratio can reach 1.5 in extreme cases (eg supersolar metallicities).  Also high-z galaxies can have Ha/[OII]=1.5 if they are metal poor (0.1Zsun) and with a highly ionized medium. The opposit (lower) limit for the ratio is 0.5. According with this range, and considering also what done by Moustakas+06, Ha/[OII]=1 is a reasonable "intermediate" value. 


*Halpha/ Hbeta*

Ha/Hb = 2.91 from McCall (1982)\
Ha/Hb = 2.85 from Storey&Hummer (1996) and from Osterbrock (1989)\
The latter value seems to me more commonly used (see e.g. Moustakas+06).


*[OIII]b/Hbeta*



This ratio spreads over a wide range. From the table in Anders & Fritze-v.Alvensleben (2003), assuming metallicity around Zsun, [OIII]b/Hbeta = 4.081. 

However, this ratio is way above other measurements and highly uncertain and scattered.
Charlot & Longhetti (2001) developed a model with [OIII]/[OII]=0.30-0.71 depending on the galaxy type. Studies on galaxy samples (like Kewley+06, Moustakas+06) show an average [OIII]b/[OII] ~ 0.3-0.4. 
So, such prediction would be a factor 3-4 below Anders et al. (2003).
We adopt [OIII]b/Hbeta = 0.4*2.85= 1.14, knowing that this value is highly uncertained.


[OIII]a at 4959A is around 1/3 of [OIII]b at 5009A.

*NII/Halpha*

NII/Halpha=0.3 from the SDSS BPT diagram 




In [ ]:
# model_name = "Ell1_A_0.sed"
model_name = "SB9_A_0.sed"
model_path = os.path.join("sed/GAL/COSMOS_SED/", model_name)

In [ ]:
config = lp.default_cosmos_config.copy()
lp.data_retrieval.get_auxiliary_data(keymap=config, additional_files=[model_path])

In [ ]:
sed_filename = os.path.join(os.path.join(os.environ["LEPHAREDIR"], model_path))
onesed = lp.GalSED("test_em_lines", 0)
onesed.read(sed_filename)

In [ ]:
fig, ax = plt.subplots()
ax.loglog(*onesed.data())
plt.xlim(800, 150020)
plt.ylim(0.00001, 0.1)
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs s$^{-1}$ Hz$^{-1}$]");

- Compute the rest-frame absolute magnitude in the NUV and r' bands
- The rest-frame NUV-R color is used to decide if the emission lines should be included, using a treshold at NUV-R=4 (passive / star-forming)

In [ ]:
onesed.compute_luminosities()
# Rest-frame absolute magnitude in the NUV
MNUV = -2.5 * onesed.luv + 51.59
# Rest-frame colors NUV-r'
NUVR = -2.5 * (onesed.luv - onesed.lopt)
print("Absolute magnitude in NUV: ", MNUV, " ; Rest-frame colors in NUV - R :", NUVR)

Compute the emission line strengths. \
This is done for one template at z=0 without dust attenuation.\
Additional steps should be done to redshift the template, apply dust attenuation, apply the opticity of the IGM, which will change the ratios.

In [ ]:
onesed.generateEmEmpUV(MNUV, NUVR)
print("Factors to be applied to each line following the empirical modelisation (lambda, factor)")
for item in onesed.fac_line:
    if item.val > 0:
        print(item.lamb, item.val)

In [ ]:
linesed = lp.GalSED(onesed)
linesed.generateEmSpectra(40)
onesed.sumSpectra(linesed, 1.0);

In [ ]:
fig, ax = plt.subplots()
ax.loglog(*onesed.data())
plt.xlim(800, 150020)
plt.ylim(0.00001, 0.1)
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs s$^{-1}$ Hz$^{-1}$]");

# Physical Model

The physical model relies on definite, even if approximate, prescriptions to evaluate the contributions of nebular emission and emission lines. It needs an SED that has coverage below ~1200. As a result, we will start by loading the Bruzual&Charlot library distributed in [lephare-data](https://github.com/lephare-photoz/lephare-data)

In [ ]:
config = lp.default_cosmos_config.copy()
lp.data_retrieval.get_auxiliary_data(
    keymap=config, additional_files=["sed/GAL/BC03_CHAB/bc2003_lr_m42_chab_tau01_dust00.ised_ASCII"]
)

In [ ]:
seds = lp.readBC03(
    os.path.join(os.environ["LEPHAREDIR"], "sed/GAL/BC03_CHAB/bc2003_lr_m42_chab_tau01_dust00.ised_ASCII"),
    0,  # id of the template
    [-999, -999],  # dummy values if no age range is requested
)

In [ ]:
fig, ax = plt.subplots()
[ax.loglog(*sed.data()) for sed in seds]
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs s$^{-1}$ Hz$^{-1}$]");

## Computing the number of ionizing photons

We first need to compute the number flux of photons able to ionize the hyddrogen atom. The code `SED::calc_ph` actually stores the number flux for four ionization edges: HeII, HeI, H, and H2 first excitation state. For a given SED, this amounts to compute the integral
$\int_0^{w_i} SED(\lambda)\cdot \frac{\lambda}{hc}\,d\lambda,$
where $w_i=\,227.8,\, 505.6,\, 911.6$, and $1108.7$ A for HeII, HeI, H, and H2 respectively, and where $hc$ is in ergs.A. 
This normalization assumes that the SED is only self consistent for sed provided in ergs/cm2/s/A, up to an overall scaling factor obtained during a fit to a set of photometric measurements. 
Results are stored in the $q_i$ array member of size 4 of the SED instance.


In [ ]:
sed = seds[20]  # we take any sed but seds[0] which is empty
print(f"SED age: {sed.age}")
sed.calc_ph()
print(f"Number flux of ionizing photons for HeII, HeI, H, and H2: {sed.qi}")

## Estimating the nebular continuum contribution

Stellar population synthesis code usually do not account for nebular emission in synthesizing a galaxy spectrum. 
Nebular emission comes from HII regions ionized by very energetic photons ($h\nu\geq 13.6$ eV), typically coming from young blue stars.
For typical HII regions (temperature in the range 5000 to 20000 K and density below $10^4$cm$^{-3}$), collisional deexcitations are negligible compared to radiative deexcitation. Recombination then yields a continum and line emission that needs to be taken into account in a photometric fit, as determined e.g. by [Shaerer and de Barros 2009](https://ui.adsabs.harvard.edu/abs/2009A%26A...502..423S/abstract).

Of course a software like `LePHARE` cannot implement a full treatment of recombination processes. It thus follows the treatment of [Shearer and Vacca 1998](https://ui.adsabs.harvard.edu/abs/1998ApJ...497..618S/abstract). We consider only "case B" nebular emission, which means that the nebula is opaque to recombination photons (and thus we are in a ionization-recombination equilibrium), with an electron density and temperature set to $n_e=100$cm$^{-3}$ and $T_e=10000\,$K
the monochromatic luminosity can be written
$$
L_\gamma = \frac{c}{\lambda^2}\frac{\gamma_{total}}{\alpha_B}f_\gamma Q_0
$$
where:
- $\gamma_{total}$ is the total nebular continuum emission coefficient, that we discuss next.
- $\alpha_B$ is the "case B" recombination coefficient for Hydrogen. Its value, $2.59\, 10^{-13}$ is taken from table 2.1 of the [Osterbrock and Ferland 2006 mononograph](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://dokumen.pub/astrophysics-of-gaseous-nebulae-and-active-galactic-nuclei-2nbsped-1891389343-9781891389344-z-6872184.html&ved=2ahUKEwjjz7eogpuRAxWdK_sDHUCDKy8QFnoECBgQAQ&usg=AOvVaw3TPx598-tx0Mxi9-dDdHK0), and is valid for $T_e=10000$K.
- $Q_0$ is the number of ionizing photons (the Lyman continuum) computed above, and $f_{\gamma}$ is the fraction of these photons absorbed by the gas; it is set to 1. in the code.

For a He abundance of 10% by number relative to hydrogen, and neglecting HeIII contribution, $\gamma_{total}$ reads:
$$
\gamma_{total} = \gamma_\nu(HI) + 0.1\,\gamma_\nu(HeI) + \gamma_\nu(2q)\quad ,
$$
where the last contribution comes from the 2 photon deexcitation of H 2 2s channel. The values of the three $\gamma$ coefficient are tabulated in `emission_lines.h` and are assembled as follows:
- [Aller 1984](https://link.springer.com/book/10.1007/978-94-010-9639-3) provides in table 4.9 values for the coefficients at $T_e=10000_,$K from $\lambda=1000\,$A to $\lambda=10000.1\,$A, in units of $10^{-40}\,$ergs cm$^3$ s$^{-1}$ Hz$^{-1}$.
- [Ferland 1980](https://articles.adsabs.harvard.edu/pdf/1980PASP...92..596F) supplement these with values for HI between 14591 and 132173 A.
- We assume $\gamma_\nu(HeI)= \gamma_\nu(HI)$  and $\gamma_\nu(2q) = 0$ above 10000.1 A.

We can access and plot the resulting curves:

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(lp._ga_lamb, lp._ga_H_val, label=r"$\gamma_\nu(HI)$")
ax.plot(lp._ga_lamb, lp._ga_2q_val, label=r"$\gamma_\nu(2q)$")
ax.plot(lp._ga_lamb, np.array(lp._ga_HeI_val), label=r"$\gamma_\nu(HeI)$")
ax.plot(
    lp._ga_lamb,
    np.array(lp._ga_H_val) + np.array(lp._ga_2q_val) + 0.1 * np.array(lp._ga_HeI_val),
    label=r"$\gamma_{tot}$",
)
plt.ylabel("emission coefficient [$10^{-40}$erg cm$^3$ s$^{-1}$ Hz$^{-1}$]")
plt.xlabel(r"$\lambda$ [A]")
plt.legend();

The resulting effect on an SED can now be illustrated

In [ ]:
print(f"galaxy age: {sed.age} yr")
x, y = sed.data()
cont = sed.add_neb_cont(sed.qi[2])
x2, y2 = sed.data()
fig, ax = plt.subplots()
ax.semilogy(x, y, label="stellar continuum")
ax.semilogy(x2, y2, label="total continuum")
ax.semilogy(x2, cont, label="nebular continuum")
plt.xlim(0, 10000)
plt.ylim(1.0e-12, 1.0e-8)
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs $s^{-1}$ Hz$^{-1}$]")
plt.legend();

## Computing the emission line contribution 

Emission line recombination luminosities are compiled from two references :
- The hydrogen Balmer Paschen and Bracket recombination lines can be found on table 4.2 of [Osterbrock and Ferland 2006 mononograph](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://dokumen.pub/astrophysics-of-gaseous-nebulae-and-active-galactic-nuclei-2nbsped-1891389343-9781891389344-z-6872184.html&ved=2ahUKEwjjz7eogpuRAxWdK_sDHUCDKy8QFnoECBgQAQ&usg=AOvVaw3TPx598-tx0Mxi9-dDdHK0)
- Non hydrogen emission lines come from [Anders and Fritze-v. Alvensleben 2003](https://ui.adsabs.harvard.edu/abs/2003A%26A...401.1063A/abstract)
In both cases, the luminosities are provided in units of the H$\beta$ luminosity, which can be computed as

$$
L_{H_\beta} = \frac{hc}{\lambda_{H_\beta}}\frac{\alpha^{eff}_{H_\beta}}{\alpha_B}f_\gamma Q_0
$$
where $\alpha^{eff}_{H_\beta}=3.03\,10^{-14}$ cm$^3$s$^{-1}$ for $T_e=10000\,$K and case B (table 4.2 in Osterbrock and Ferland again).

We can visualize the lines in the following way

In [ ]:
sed.generateEmPhys(sed.zmet, sed.qi[2])
x = [onel.lamb for onel in sed.fac_line]
y = [onel.val for onel in sed.fac_line]
fig, ax = plt.subplots()
for xx, yy in zip(x, y):
    ax.plot([xx, xx], [0, yy], "-")

These are then transformed into gaussians using an arbitrary 100km/s line dispersion, 40 steps and a span of 6 times the line dispersion.

In [ ]:
linesed = lp.GalSED(sed)
linesed.generateEmSpectra(40)
fig, ax = plt.subplots()
ax.plot(*linesed.data())
plt.xlim(0, 40000)
# plt.ylim(0, 1.e-7)